Task 1: Third-order letter approximation model

In [228]:
files = [
    '/workspaces/emergingtechnologies/Data/Frankenstein.txt',
    '/workspaces/emergingtechnologies/Data/BookPlates.txt',
    '/workspaces/emergingtechnologies/Data/SpyglassMoutain.txt',
    '/workspaces/emergingtechnologies/Data/TheOldHouse.txt',
    '/workspaces/emergingtechnologies/Data/WindOfDestiny.txt',
]

texts = {}

#preamble and postamble markers
start_marker = " ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"

In [229]:
#read each file and remove preamble and postamble
for file_path in files:
    with open(file_path, 'r') as f:
        text = f.read()

        #find the start and end markers
        start = text.find(start_marker)
        end = text.find(end_marker)

        #skip the start marker
        start += len(start_marker)
        #extract all text between the markers
        text = text[start:end].strip()

        #store the cleaned text in the dictionary with filename as the key
        filename = file_path.split('/')[-1]  # Get filename from path
        texts[filename] = text

In [230]:
for filename, text in texts.items():
    #remove all newlines
    text = text.replace('\n', '')    
    chars = set('ABCDEFGHIJKLMNOPQRSTUVWXYZ. ')
    text = text.upper()
    #remove all characters that are not in the set of allowed characters
    cleaned = ''.join(c for c in text if c in chars)
    #store the cleaned text back in the dictionary
    texts[filename] = cleaned

print(texts['BookPlates.txt'])

PRODUCED BY CHRIS CURNOW EMMY AND THE ONLINE DISTRIBUTEDPROOFREADING TEAM AT HTTPWWW.PGDP.NETTRANSCRIBERS NOTES TWO LETTERS HAD MACRONS ABOVE THEM IN THE ORGINALTHESE HAVE BEEN MARKED AS I AND M.A CAROT  BEFORE BRACKETED LETTERS INDICATED THAT THE LETTER OR LETTERSWERE SUPERSCRIPTED IN THE ORGINAL HONBLE.ILLUSTRATIONBOOKPLATESBY W. J. HARDY F.S.A.SECOND EDITIONILLUSTRATION    LONDON    KEGAN PAUL TRENCH TRBNER  CO. LTD.    MDCCCXCVII    FIRST EDITION PUBLISHED  AS VOL. II. OF BOOKS ABOUT BOOKS.PREFACEHAVING VINDICATED IN MY INTRODUCTORY CHAPTER THE PRACTICE OF COLLECTINGBOOKPLATES FROM THE CHARGE OF FLAGRANT IMMORALITY I DO NOT THINK ITNECESSARY TO SPEND MANY WORDS IN DEMONSTRATING THAT IT IS IN EVERY WAY AWORTHY AND REASONABLE PURSUIT AND ONE WHICH FULLY DESERVES TO BE MADETHE SUBJECT OF A SPECIAL TREATISE IN A SERIES OF BOOKS ABOUT BOOKS. IFNEED WERE THE EDITOR OF THE SERIES WHO ASKED ME TO WRITE THIS LITTLEHANDBOOK WOULD PERHAPS KINDLY ACCEPT HIS SHARE OF RESPONSIBILITY BUTIN THE FA

In [231]:
model = {}

for filename, cleaned in texts.items():
  #iterate over the text and count the occurrences of each trigram
  for i in range(len(cleaned) - 1):
    #extract the trigram
    trigram = text[i:i+3]
    if trigram != '' and trigram != '   ':
        #store the trigram in the model
      model[trigram] = model.get(trigram, 0) + 1

In [232]:
sorted(model.items(), key=lambda x: x[1], reverse=True)

[(' TH', 10344),
 ('THE', 8433),
 ('HE ', 7227),
 (' I ', 3801),
 ('AND', 3794),
 (' AN', 3727),
 ('ND ', 3695),
 (' TO', 3674),
 ('ING', 3284),
 ('TO ', 3174),
 ('ED ', 3073),
 ('AT ', 2871),
 (' OF', 2698),
 ('NG ', 2659),
 ('OF ', 2575),
 ('ER ', 2569),
 (' A ', 2430),
 (' HA', 2427),
 (' IN', 2421),
 ('YOU', 2362),
 (' YO', 2343),
 ('HAT', 2341),
 ('IN ', 2253),
 ('HER', 2197),
 ('E T', 2163),
 (' HE', 2159),
 ('THA', 2116),
 ('AS ', 2038),
 (' WA', 1882),
 ('N T', 1820),
 ('IS ', 1795),
 ('T T', 1735),
 ('EN ', 1735),
 (' WH', 1698),
 ('RE ', 1640),
 ('E S', 1608),
 ('D T', 1605),
 (' LI', 1594),
 (' IT', 1586),
 ('. I', 1571),
 (' BE', 1534),
 ('VE ', 1524),
 ('ME ', 1490),
 (' ME', 1482),
 ('MY ', 1458),
 ('E W', 1450),
 (' MY', 1429),
 ('ERE', 1425),
 (' WI', 1391),
 ('UT ', 1390),
 ('IT ', 1390),
 ('T I', 1381),
 (' ON', 1336),
 ('ON ', 1335),
 ('E A', 1333),
 ('S A', 1324),
 ('AVE', 1319),
 (' CO', 1287),
 ('TER', 1284),
 ('OU ', 1276),
 ('GHT', 1267),
 ('LL ', 1251),
 ('S T'